In [ ]:
import pandas as pd

def clean_data(df):
    # Drop columns: 'gameid', 'datacompleteness' and 50 other columns
    df = df.drop(columns=['gameid', 'datacompleteness', 'url', 'league', 'split', 'year', 'playoffs', 'date', 'game', 'patch', 'participantid', 'side', 'teamid', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'gamelength', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'firstbloodassist', 'firstbloodkill', 'firstbloodvictim', 'firstdragon', 'dragons', 'opp_dragons', 'elementaldrakes', 'infernals', 'opp_elementaldrakes', 'mountains', 'clouds', 'oceans', 'chemtechs', 'hextechs', 'dragons (type unknown)', 'elders', 'opp_elders', 'firstherald', 'heralds', 'opp_heralds', 'firstbaron', 'firsttower', 'towers', 'opp_towers', 'firstmidtower', 'firsttothreetowers', 'turretplates', 'opp_turretplates'])
    # Drop rows with missing data in columns: 'playername', 'result'
    df = df.dropna(subset=['playername', 'result'])
    # Drop column: 'gspd'
    df = df.drop(columns=['gspd'])
    # Drop column: 'monsterkillsenemyjungle'
    df = df.drop(columns=['monsterkillsenemyjungle'])
    # Drop column: 'monsterkillsownjungle'
    df = df.drop(columns=['monsterkillsownjungle'])
    # Drop rows with missing data across all columns
    df = df.dropna()
    # Drop column: 'playerid'
    df = df.drop(columns=['playerid'])
    # Export DataFrame to an Excel file
    df.to_excel("output.xlsx", index=False)
    return df

# Loaded variable 'df' from URI: c:\Users\ssjed\OneDrive\Documents\GitHub\Fall2024\STAT 335\Final\2024_LoL_esports_match_data_from_OraclesElixir.xlsx
df = pd.read_excel(r'c:\Users\ssjed\OneDrive\Documents\GitHub\Fall2024\STAT 335\Final\2024_LoL_esports_match_data_from_OraclesElixir.xlsx')

df_clean = clean_data(df.copy())
df_clean.head()


In [ ]:
import numpy as np
import random

# Set the seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)

In [ ]:
# Select only numeric columns
numeric_df_clean = df_clean.select_dtypes(include=['number'])

# Calculate the correlation matrix
correlation_matrix = numeric_df_clean.corr()

# Get the absolute values of the correlation matrix
abs_correlation_matrix = correlation_matrix.abs()

# Unstack the matrix and sort by correlation value
sorted_correlations = abs_correlation_matrix.unstack().sort_values(ascending=False)

# Drop the duplicate pairs (correlation of a variable with itself)
sorted_correlations = sorted_correlations[sorted_correlations != 1]

# Display the sorted correlations
sorted_correlations

# Calculate the correlation of 'result' with all other columns
result_correlations = abs_correlation_matrix['result']

# Sort the correlations by absolute value in descending order
sorted_result_correlations = result_correlations.sort_values(ascending=False)

# Display the sorted correlations
sorted_result_correlations

In [ ]:
# Select only numeric columns
numeric_df_clean = df_clean.select_dtypes(include=['number'])

# Calculate the correlation matrix
correlation_matrix = numeric_df_clean.corr()

# Get the absolute values of the correlation matrix
abs_correlation_matrix = correlation_matrix.abs()

# Unstack the matrix and sort by correlation value
sorted_correlations = abs_correlation_matrix.unstack().sort_values(ascending=False)

# Drop the duplicate pairs (correlation of a variable with itself)
sorted_correlations = sorted_correlations[sorted_correlations != 1]

# Display the sorted correlations
sorted_correlations

In [ ]:
# Create datasets based on position
positions = ['top', 'jng', 'mid', 'bot', 'sup']
datasets = {}

for position in positions:
    datasets[position] = df_clean[df_clean['position'] == position]

# Display the first few rows of each dataset
for position, dataset in datasets.items():
    print(f"Dataset for position: {position}")
    print(dataset.head())
    print("\n")

RUN ON FIRST TIME ONLY

In [ ]:
# df_clean_teams = df_clean.sort_values(['teamname'], ascending=[False])
# df_clean_teams.head()
# df_clean_teams.to_excel("teams.xlsx", index=False)


In [ ]:
# import pandas as pd
# import os

# # Create an Excel file for each role
# for position, dataset in datasets.items():
#     filename = f"{position}_dataset.xlsx"
#     dataset.to_excel(filename, index=False)
#     print(f"Created {filename}")

RUN ON FIRST TIME ONLY

In [ ]:
# # Create the 'roles' directory if it doesn't exist
# if not os.path.exists('roles'):
#     os.makedirs('roles')

# # Move the Excel files to the 'roles' directory
# for position, dataset in datasets.items():
#     filename = f"{position}_dataset.xlsx"
#     destination = os.path.join('roles', filename)
#     if os.path.exists(destination):
#         os.remove(destination)
#     os.rename(filename, destination)
#     print(f"Moved {filename} to roles/{filename}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Define the feature columns and target variable
features = numeric_df_clean.drop(columns=['result'])
target = numeric_df_clean['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create and train the logistic regression model
model = LogisticRegression(max_iter=8334)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

In [ ]:
import numpy as np

import statsmodels.api as sm

# Add a constant to the features
X_train_const = sm.add_constant(X_train)

# Fit the initial model with all features
model = sm.Logit(y_train, X_train_const).fit()

# Perform backward selection
def backward_selection(data, target, significance_level=0.05):
    features = data.columns.tolist()
    while len(features) > 0:
        X = sm.add_constant(data[features])
        model = sm.Logit(target, X).fit(disp=0)
        p_values = model.pvalues[1:]  # Exclude the intercept
        max_p_value = p_values.max()
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break
    return features

# Run backward selection
selected_features = backward_selection(X_train, y_train)

# Fit the final model with selected features
X_train_selected = sm.add_constant(X_train[selected_features])
final_model = sm.Logit(y_train, X_train_selected).fit()

# Evaluate the final model
X_test_selected = sm.add_constant(X_test[selected_features])
y_pred = final_model.predict(X_test_selected)
y_pred_class = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_class)
report = classification_report(y_test, y_pred_class)

print(f'Selected Features: {selected_features}')
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

In [ ]:
# Load the teams dataset
df_teams = pd.read_excel("teams.xlsx")

# Select only numeric columns
numeric_df_teams = df_teams.select_dtypes(include=['number'])

# Define the feature columns and target variable
features_teams = numeric_df_teams.drop(columns=['result'])
target_teams = numeric_df_teams['result']

# Split the data into training and testing sets
X_train_teams, X_test_teams, y_train_teams, y_test_teams = train_test_split(features_teams, target_teams, test_size=0.2, random_state=42)

# Create and train the logistic regression model
model_teams = LogisticRegression(max_iter=8334)
model_teams.fit(X_train_teams, y_train_teams)

# Make predictions on the test set
y_pred_teams = model_teams.predict(X_test_teams)

# Evaluate the model
accuracy_teams = accuracy_score(y_test_teams, y_pred_teams)
report_teams = classification_report(y_test_teams, y_pred_teams)

print(f'Accuracy: {accuracy_teams}')
print('Classification Report:')
print(report_teams)

In [ ]:
# Run backward selection for the teams dataset
selected_features_teams = backward_selection(features_teams, target_teams)

# Fit the final model with selected features
X_train_teams_selected = sm.add_constant(X_train_teams[selected_features_teams])
final_model_teams = sm.Logit(y_train_teams, X_train_teams_selected).fit()

# Evaluate the final model
X_test_teams_selected = sm.add_constant(X_test_teams[selected_features_teams])
y_pred_teams_selected = final_model_teams.predict(X_test_teams_selected)
y_pred_teams_class = (y_pred_teams_selected > 0.5).astype(int)

accuracy_teams_selected = accuracy_score(y_test_teams, y_pred_teams_class)
report_teams_selected = classification_report(y_test_teams, y_pred_teams_class)

print(f'Selected Features: {selected_features_teams}')
print(f'Accuracy: {accuracy_teams_selected}')
print('Classification Report:')
print(report_teams_selected)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Iterate through each dataset in the roles folder
for position, dataset in datasets.items():
    # Select only numeric columns
    numeric_dataset = dataset.select_dtypes(include=['number'])
    
    # Define the feature columns and target variable
    features = numeric_dataset.drop(columns=['result'])
    target = numeric_dataset['result']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    # Create and train the logistic regression model
    model = LogisticRegression(max_iter=8334)
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f'Position: {position}')
    print(f'Accuracy: {accuracy}')
    print('Classification Report:')
    print(report)
    print('\n')

In [ ]:
# Dictionary to store the best models and their summaries for each role
best_models = {}

# Iterate through each dataset in the roles folder
for position, dataset in datasets.items():
    # Select only numeric columns
    numeric_dataset = dataset.select_dtypes(include=['number'])
    
    # Define the feature columns and target variable
    features = numeric_dataset.drop(columns=['result'])
    target = numeric_dataset['result']
    
    # Run backward selection
    selected_features = backward_selection(features, target)
    
    # Fit the final model with selected features
    X_train_selected = sm.add_constant(features[selected_features])
    final_model = sm.Logit(target, X_train_selected).fit()
    
    # Save the selected features and model summary
    best_models[position] = {
        'selected_features': selected_features,
        'model_summary': final_model.summary()
    }
    
    # Display the selected features and model summary
    print(f'Position: {position}')
    print(f'Selected Features: {selected_features}')
    print(final_model.summary())
    print('\n')

# Display the best models and their summaries for each role
for position, model_info in best_models.items():
    print(f'Position: {position}')
    print(f'Selected Features: {model_info["selected_features"]}')
    print(model_info['model_summary'])
    print('\n')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define the feature columns and target variable
features = numeric_df_clean.drop(columns=['result'])
target = numeric_df_clean['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
log_accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {log_accuracy}')
print('Classification Report:')
print(report)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Function to perform backward selection
def backward_selection_rf(X_train, y_train, X_test, y_test, significance_level=0.05):
    features = X_train.columns.tolist()
    best_accuracy = 0
    best_features = features.copy()
    
    while len(features) > 0:
        # Train the Random Forest model
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_model.fit(X_train[features], y_train)
        
        # Make predictions on the test set
        y_pred = rf_model.predict(X_test[features])
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_features = features.copy()
        
        # Get feature importances
        importances = rf_model.feature_importances_
        
        # Find the least important feature
        least_important_feature = features[np.argmin(importances)]
        
        # Remove the least important feature
        features.remove(least_important_feature)
    
    return best_features, best_accuracy

# Run backward selection
selected_features_rf, best_accuracy_rf = backward_selection_rf(X_train, y_train, X_test, y_test)

# Fit the final Random Forest model with selected features
rf_model_final = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_final.fit(X_train[selected_features_rf], y_train)

# Make predictions on the test set
y_pred_rf_final = rf_model_final.predict(X_test[selected_features_rf])

# Evaluate the final model
accuracy_rf_final = accuracy_score(y_test, y_pred_rf_final)
report_rf_final = classification_report(y_test, y_pred_rf_final)

print(f'Selected Features: {selected_features_rf}')
print(f'Best Accuracy: {best_accuracy_rf}')
print('Final Model Accuracy:', accuracy_rf_final)
print('Classification Report:')
print(report_rf_final)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define the feature columns and target variable
features_teams = numeric_df_teams.drop(columns=['result'])
target_teams = numeric_df_teams['result']

# Split the data into training and testing sets
X_train_teams, X_test_teams, y_train_teams, y_test_teams = train_test_split(features_teams, target_teams, test_size=0.2, random_state=42)

# Create and train the Random Forest model
rf_model_teams = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_teams.fit(X_train_teams, y_train_teams)

# Make predictions on the test set
y_pred_teams = rf_model_teams.predict(X_test_teams)

# Evaluate the model
accuracy_teams = accuracy_score(y_test_teams, y_pred_teams)
report_teams = classification_report(y_test_teams, y_pred_teams)

print(f'Accuracy: {accuracy_teams}')
print('Classification Report:')
print(report_teams)

In [ ]:
# Run backward selection for the teams dataset using Random Forest
selected_features_rf_teams, best_accuracy_rf_teams = backward_selection_rf(X_train_teams, y_train_teams, X_test_teams, y_test_teams)

# Fit the final Random Forest model with selected features
rf_model_final_teams = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_final_teams.fit(X_train_teams[selected_features_rf_teams], y_train_teams)

# Make predictions on the test set
y_pred_rf_final_teams = rf_model_final_teams.predict(X_test_teams[selected_features_rf_teams])

# Evaluate the final model
accuracy_rf_final_teams = accuracy_score(y_test_teams, y_pred_rf_final_teams)
report_rf_final_teams = classification_report(y_test_teams, y_pred_rf_final_teams)

print(f'Selected Features: {selected_features_rf_teams}')
print(f'Best Accuracy: {best_accuracy_rf_teams}')
print('Final Model Accuracy:', accuracy_rf_final_teams)
print('Classification Report:')
print(report_rf_final_teams)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Iterate through each dataset in the roles folder
for position, dataset in datasets.items():
    # Select only numeric columns
    numeric_dataset = dataset.select_dtypes(include=['number'])
    
    # Define the feature columns and target variable
    features = numeric_dataset.drop(columns=['result'])
    target = numeric_dataset['result']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    # Create and train the Random Forest model
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = rf_model.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f'Position: {position}')
    print(f'Accuracy: {accuracy}')
    print('Classification Report:')
    print(report)
    print('\n')

In [ ]:
# Dictionary to store the best models and their accuracies for each role
best_models_rf = {}

# Iterate through each dataset in the roles folder
for position, dataset in datasets.items():
    # Select only numeric columns
    numeric_dataset = dataset.select_dtypes(include=['number'])
    
    # Define the feature columns and target variable
    features = numeric_dataset.drop(columns=['result'])
    target = numeric_dataset['result']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    # Run backward selection
    selected_features_rf, best_accuracy_rf = backward_selection_rf(X_train, y_train, X_test, y_test)
    
    # Fit the final Random Forest model with selected features
    rf_model_final = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model_final.fit(X_train[selected_features_rf], y_train)
    
    # Make predictions on the test set
    y_pred_rf_final = rf_model_final.predict(X_test[selected_features_rf])
    
    # Evaluate the final model
    accuracy_rf_final = accuracy_score(y_test, y_pred_rf_final)
    report_rf_final = classification_report(y_test, y_pred_rf_final)
    
    # Save the best model and its accuracy
    best_models_rf[position] = {
        'selected_features': selected_features_rf,
        'best_accuracy': best_accuracy_rf,
        'final_model_accuracy': accuracy_rf_final,
        'classification_report': report_rf_final
    }
    
    print(f'Position: {position}')
    print(f'Selected Features: {selected_features_rf}')
    print(f'Best Accuracy: {best_accuracy_rf}')
    print('Final Model Accuracy:', accuracy_rf_final)
    print('Classification Report:')
    print(report_rf_final)
    print('\n')

# Display the best models and their accuracies for each role
for position, model_info in best_models_rf.items():
    print(f'Position: {position}')
    print(f'Selected Features: {model_info["selected_features"]}')
    print(f'Best Accuracy: {model_info["best_accuracy"]}')
    print(f'Final Model Accuracy: {model_info["final_model_accuracy"]}')
    print('Classification Report:')
    print(model_info['classification_report'])
    print('\n')

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error

# Define the feature columns and target variable
features = numeric_df_clean.drop(columns=['result'])
target = numeric_df_clean['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create and train the LASSO regression model
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_lasso = lasso_model.predict(X_test)

# Evaluate the LASSO model
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
print(f'LASSO Regression Mean Squared Error: {mse_lasso}')

# Create and train the Ridge regression model
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_ridge = ridge_model.predict(X_test)

# Evaluate the Ridge model
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
print(f'Ridge Regression Mean Squared Error: {mse_ridge}')


In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error

# Define the feature columns and target variable
features_teams = numeric_df_teams.drop(columns=['result'])
target_teams = numeric_df_teams['result']

# Split the data into training and testing sets
X_train_teams, X_test_teams, y_train_teams, y_test_teams = train_test_split(features_teams, target_teams, test_size=0.2, random_state=42)

# Create and train the LASSO regression model
lasso_model_teams = Lasso(alpha=0.1)
lasso_model_teams.fit(X_train_teams, y_train_teams)

# Make predictions on the test set
y_pred_lasso_teams = lasso_model_teams.predict(X_test_teams)

# Evaluate the LASSO model
mse_lasso_teams = mean_squared_error(y_test_teams, y_pred_lasso_teams)
print(f'LASSO Regression Mean Squared Error for Teams: {mse_lasso_teams}')

# Create and train the Ridge regression model
ridge_model_teams = Ridge(alpha=1.0)
ridge_model_teams.fit(X_train_teams, y_train_teams)

# Make predictions on the test set
y_pred_ridge_teams = ridge_model_teams.predict(X_test_teams)

# Evaluate the Ridge model
mse_ridge_teams = mean_squared_error(y_test_teams, y_pred_ridge_teams)
print(f'Ridge Regression Mean Squared Error for Teams: {mse_ridge_teams}')

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error


# Iterate through each dataset in the roles folder
for position, dataset in datasets.items():
    # Select only numeric columns
    numeric_dataset = dataset.select_dtypes(include=['number'])
    
    # Define the feature columns and target variable
    features = numeric_dataset.drop(columns=['result'])
    target = numeric_dataset['result']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    # Create and train the LASSO regression model
    lasso_model = Lasso(alpha=0.1)
    lasso_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_lasso = lasso_model.predict(X_test)
    
    # Evaluate the LASSO model
    mse_lasso = mean_squared_error(y_test, y_pred_lasso)
    
    # Create and train the Ridge regression model
    ridge_model = Ridge(alpha=1.0)
    ridge_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_ridge = ridge_model.predict(X_test)
    
    # Evaluate the Ridge model
    mse_ridge = mean_squared_error(y_test, y_pred_ridge)
    
    print(f'Position: {position}')
    print(f'LASSO Regression Mean Squared Error: {mse_lasso}')
    print(f'Ridge Regression Mean Squared Error: {mse_ridge}')
    print('\n')
    # Save the models and their mean squared errors
    best_models[position] = {
        'lasso_model': lasso_model,
        'ridge_model': ridge_model,
        'mse_lasso': mse_lasso,
        'mse_ridge': mse_ridge
    }

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define the feature columns and target variable
features = numeric_df_clean.drop(columns=['result'])
target = numeric_df_clean['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create and train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_knn = knn_model.predict(X_test)

# Evaluate the model
accuracy_knn = accuracy_score(y_test, y_pred_knn)
report_knn = classification_report(y_test, y_pred_knn)

print(f'Accuracy: {accuracy_knn}')
print('Classification Report:')
print(report_knn)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Function to perform backward selection
def backward_selection_knn(X_train, y_train, X_test, y_test, significance_level=0.05):
    features = X_train.columns.tolist()
    best_accuracy = 0
    best_features = features.copy()
    
    while len(features) > 0:
        # Train the KNN model
        knn_model = KNeighborsClassifier(n_neighbors=5)
        knn_model.fit(X_train[features], y_train)
        
        # Make predictions on the test set
        y_pred = knn_model.predict(X_test[features])
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_features = features.copy()
        
        # Calculate feature importances (using permutation importance)
        importances = []
        for feature in features:
            X_test_permuted = X_test.copy()
            X_test_permuted[feature] = np.random.permutation(X_test[feature])
            y_pred_permuted = knn_model.predict(X_test_permuted[features])
            permuted_accuracy = accuracy_score(y_test, y_pred_permuted)
            importances.append(best_accuracy - permuted_accuracy)
        
        # Find the least important feature
        least_important_feature = features[np.argmin(importances)]
        
        # Remove the least important feature
        features.remove(least_important_feature)
    
    return best_features, best_accuracy

# Run backward selection
selected_features_knn, best_accuracy_knn = backward_selection_knn(X_train, y_train, X_test, y_test)

# Fit the final KNN model with selected features
knn_model_final = KNeighborsClassifier(n_neighbors=5)
knn_model_final.fit(X_train[selected_features_knn], y_train)

# Make predictions on the test set
y_pred_knn_final = knn_model_final.predict(X_test[selected_features_knn])

# Evaluate the final model
accuracy_knn_final = accuracy_score(y_test, y_pred_knn_final)
report_knn_final = classification_report(y_test, y_pred_knn_final)

print(f'Selected Features: {selected_features_knn}')
print(f'Best Accuracy: {best_accuracy_knn}')
print('Final Model Accuracy:', accuracy_knn_final)
print('Classification Report:')
print(report_knn_final)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define the feature columns and target variable
features_teams = numeric_df_teams.drop(columns=['result'])
target_teams = numeric_df_teams['result']

# Split the data into training and testing sets
X_train_teams, X_test_teams, y_train_teams, y_test_teams = train_test_split(features_teams, target_teams, test_size=0.2, random_state=42)

# Create and train the KNN model
knn_model_teams = KNeighborsClassifier(n_neighbors=5)
knn_model_teams.fit(X_train_teams, y_train_teams)

# Make predictions on the test set
y_pred_teams_knn = knn_model_teams.predict(X_test_teams)

# Evaluate the model
accuracy_teams_knn = accuracy_score(y_test_teams, y_pred_teams_knn)
report_teams_knn = classification_report(y_test_teams, y_pred_teams_knn)

print(f'Accuracy: {accuracy_teams_knn}')
print('Classification Report:')
print(report_teams_knn)

In [ ]:
# Run backward selection for the teams dataset using KNN
selected_features_knn_teams, best_accuracy_knn_teams = backward_selection_knn(X_train_teams, y_train_teams, X_test_teams, y_test_teams)

# Fit the final KNN model with selected features
knn_model_final_teams = KNeighborsClassifier(n_neighbors=5)
knn_model_final_teams.fit(X_train_teams[selected_features_knn_teams], y_train_teams)

# Make predictions on the test set
y_pred_knn_final_teams = knn_model_final_teams.predict(X_test_teams[selected_features_knn_teams])

# Evaluate the final model
accuracy_knn_final_teams = accuracy_score(y_test_teams, y_pred_knn_final_teams)
report_knn_final_teams = classification_report(y_test_teams, y_pred_knn_final_teams)

print(f'Selected Features: {selected_features_knn_teams}')
print(f'Best Accuracy: {best_accuracy_knn_teams}')
print('Final Model Accuracy:', accuracy_knn_final_teams)
print('Classification Report:')
print(report_knn_final_teams)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Iterate through each dataset in the roles folder
for position, dataset in datasets.items():
    # Select only numeric columns
    numeric_dataset = dataset.select_dtypes(include=['number'])
    
    # Define the feature columns and target variable
    features = numeric_dataset.drop(columns=['result'])
    target = numeric_dataset['result']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    # Create and train the KNN model
    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = knn_model.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f'Position: {position}')
    print(f'Accuracy: {accuracy}')
    print('Classification Report:')
    print(report)
    print('\n')

In [ ]:
# Dictionary to store the best models and their accuracies for each role
best_models_knn = {}

# Iterate through each dataset in the roles folder
for position, dataset in datasets.items():
    # Select only numeric columns
    numeric_dataset = dataset.select_dtypes(include=['number'])
    
    # Define the feature columns and target variable
    features = numeric_dataset.drop(columns=['result'])
    target = numeric_dataset['result']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    # Run backward selection
    selected_features_knn, best_accuracy_knn = backward_selection_knn(X_train, y_train, X_test, y_test)
    
    # Fit the final KNN model with selected features
    knn_model_final = KNeighborsClassifier(n_neighbors=5)
    knn_model_final.fit(X_train[selected_features_knn], y_train)
    
    # Make predictions on the test set
    y_pred_knn_final = knn_model_final.predict(X_test[selected_features_knn])
    
    # Evaluate the final model
    accuracy_knn_final = accuracy_score(y_test, y_pred_knn_final)
    report_knn_final = classification_report(y_test, y_pred_knn_final)
    
    # Save the best model and its accuracy
    best_models_knn[position] = {
        'selected_features': selected_features_knn,
        'best_accuracy': best_accuracy_knn,
        'final_model_accuracy': accuracy_knn_final,
        'classification_report': report_knn_final
    }
    
    print(f'Position: {position}')
    print(f'Selected Features: {selected_features_knn}')
    print(f'Best Accuracy: {best_accuracy_knn}')
    print('Final Model Accuracy:', accuracy_knn_final)
    print('Classification Report:')
    print(report_knn_final)
    print('\n')

# Display the best models and their accuracies for each role
for position, model_info in best_models_knn.items():
    print(f'Position: {position}')
    print(f'Selected Features: {model_info["selected_features"]}')
    print(f'Best Accuracy: {model_info["best_accuracy"]}')
    print(f'Final Model Accuracy: {model_info["final_model_accuracy"]}')
    print('Classification Report:')
    print(model_info['classification_report'])
    print('\n')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Define the feature columns and target variable
features = numeric_df_clean.drop(columns=['result'])
target = numeric_df_clean['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Make predictions on the test set
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Function to perform backward selection
def backward_selection_nn(X_train, y_train, X_test, y_test, significance_level=0.05):
    features = X_train.columns.tolist()
    best_accuracy = 0
    best_features = features.copy()
    
    while len(features) > 0:
        # Standardize the features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train[features])
        X_test_scaled = scaler.transform(X_test[features])
        
        # Create the neural network model
        model = Sequential()
        model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        
        # Compile the model
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
        # Train the model
        model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
        
        # Make predictions on the test set
        y_pred_prob = model.predict(X_test_scaled)
        y_pred = (y_pred_prob > 0.5).astype(int)
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_features = features.copy()
        
        # Calculate feature importances (using permutation importance)
        importances = []
        for feature in features:
            X_test_permuted = X_test.copy()
            X_test_permuted[feature] = np.random.permutation(X_test[feature])
            X_test_permuted_scaled = scaler.transform(X_test_permuted[features])
            y_pred_permuted_prob = model.predict(X_test_permuted_scaled)
            y_pred_permuted = (y_pred_permuted_prob > 0.5).astype(int)
            permuted_accuracy = accuracy_score(y_test, y_pred_permuted)
            importances.append(best_accuracy - permuted_accuracy)
        
        # Find the least important feature
        least_important_feature = features[np.argmin(importances)]
        
        # Remove the least important feature
        features.remove(least_important_feature)
    
    return best_features, best_accuracy

# Run backward selection
selected_features_nn, best_accuracy_nn = backward_selection_nn(X_train, y_train, X_test, y_test)

# Fit the final neural network model with selected features
scaler = StandardScaler()
X_train_scaled_selected = scaler.fit_transform(X_train[selected_features_nn])
X_test_scaled_selected = scaler.transform(X_test[selected_features_nn])

model_final_nn = Sequential()
model_final_nn.add(Dense(64, input_dim=X_train_scaled_selected.shape[1], activation='relu'))
model_final_nn.add(Dense(32, activation='relu'))
model_final_nn.add(Dense(1, activation='sigmoid'))

# Compile the model
model_final_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_final_nn.fit(X_train_scaled_selected, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Make predictions on the test set
y_pred_prob_final_nn = model_final_nn.predict(X_test_scaled_selected)
y_pred_final_nn = (y_pred_prob_final_nn > 0.5).astype(int)

# Evaluate the final model
accuracy_final_nn = accuracy_score(y_test, y_pred_final_nn)
report_final_nn = classification_report(y_test, y_pred_final_nn)

print(f'Selected Features: {selected_features_nn}')
print(f'Best Accuracy: {best_accuracy_nn}')
print('Final Model Accuracy:', accuracy_final_nn)
print('Classification Report:')
print(report_final_nn)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Define the feature columns and target variable
features_teams = numeric_df_teams.drop(columns=['result'])
target_teams = numeric_df_teams['result']

# Split the data into training and testing sets
X_train_teams, X_test_teams, y_train_teams, y_test_teams = train_test_split(features_teams, target_teams, test_size=0.2, random_state=42)

# Standardize the features
scaler_teams = StandardScaler()
X_train_teams_scaled = scaler_teams.fit_transform(X_train_teams)
X_test_teams_scaled = scaler_teams.transform(X_test_teams)

# Create the neural network model
model_teams_nn = Sequential()
model_teams_nn.add(Dense(64, input_dim=X_train_teams_scaled.shape[1], activation='relu'))
model_teams_nn.add(Dense(32, activation='relu'))
model_teams_nn.add(Dense(1, activation='sigmoid'))

# Compile the model
model_teams_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_teams_nn.fit(X_train_teams_scaled, y_train_teams, epochs=50, batch_size=32, validation_split=0.2)

# Make predictions on the test set
y_pred_teams_prob_nn = model_teams_nn.predict(X_test_teams_scaled)
y_pred_teams_nn = (y_pred_teams_prob_nn > 0.5).astype(int)

# Evaluate the model
accuracy_teams_nn = accuracy_score(y_test_teams, y_pred_teams_nn)
report_teams_nn = classification_report(y_test_teams, y_pred_teams_nn)

print(f'Accuracy: {accuracy_teams_nn}')
print('Classification Report:')
print(report_teams_nn)

In [ ]:
# Run backward selection for the teams dataset using neural network
selected_features_nn_teams, best_accuracy_nn_teams = backward_selection_nn(X_train_teams, y_train_teams, X_test_teams, y_test_teams)

# Fit the final neural network model with selected features
scaler_teams = StandardScaler()
X_train_teams_scaled_selected = scaler_teams.fit_transform(X_train_teams[selected_features_nn_teams])
X_test_teams_scaled_selected = scaler_teams.transform(X_test_teams[selected_features_nn_teams])

model_final_nn_teams = Sequential()
model_final_nn_teams.add(Dense(64, input_dim=X_train_teams_scaled_selected.shape[1], activation='relu'))
model_final_nn_teams.add(Dense(32, activation='relu'))
model_final_nn_teams.add(Dense(1, activation='sigmoid'))

# Compile the model
model_final_nn_teams.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_final_nn_teams.fit(X_train_teams_scaled_selected, y_train_teams, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Make predictions on the test set
y_pred_prob_final_nn_teams = model_final_nn_teams.predict(X_test_teams_scaled_selected)
y_pred_final_nn_teams = (y_pred_prob_final_nn_teams > 0.5).astype(int)

# Evaluate the final model
accuracy_final_nn_teams = accuracy_score(y_test_teams, y_pred_final_nn_teams)
report_final_nn_teams = classification_report(y_test_teams, y_pred_final_nn_teams)

print(f'Selected Features: {selected_features_nn_teams}')
print(f'Best Accuracy: {best_accuracy_nn_teams}')
print('Final Model Accuracy:', accuracy_final_nn_teams)
print('Classification Report:')
print(report_final_nn_teams)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Iterate through each dataset in the roles folder
for position, dataset in datasets.items():
    # Select only numeric columns
    numeric_dataset = dataset.select_dtypes(include=['number'])
    
    # Define the feature columns and target variable
    features = numeric_dataset.drop(columns=['result'])
    target = numeric_dataset['result']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Create the neural network model
    model = Sequential()
    model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
    
    # Make predictions on the test set
    y_pred_prob = model.predict(X_test_scaled)
    y_pred = (y_pred_prob > 0.5).astype(int)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f'Position: {position}')
    print(f'Accuracy: {accuracy}')
    print('Classification Report:')
    print(report)
    print('\n')

In [ ]:
# Filter the dataset to only include the 'top' position
top_dataset = datasets['top']

# Select only numeric columns
numeric_dataset = top_dataset.select_dtypes(include=['number'])

# Define the feature columns and target variable
features = numeric_dataset.drop(columns=['result'])
target = numeric_dataset['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Run backward selection
selected_features_nn, best_accuracy_nn = backward_selection_nn(X_train, y_train, X_test, y_test)

# Fit the final neural network model with selected features
scaler = StandardScaler()
X_train_scaled_selected = scaler.fit_transform(X_train[selected_features_nn])
X_test_scaled_selected = scaler.transform(X_test[selected_features_nn])

model_final_nn = Sequential()
model_final_nn.add(Dense(64, input_dim=X_train_scaled_selected.shape[1], activation='relu'))
model_final_nn.add(Dense(32, activation='relu'))
model_final_nn.add(Dense(1, activation='sigmoid'))

# Compile the model
model_final_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_final_nn.fit(X_train_scaled_selected, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Make predictions on the test set
y_pred_prob_final_nn = model_final_nn.predict(X_test_scaled_selected)
y_pred_final_nn = (y_pred_prob_final_nn > 0.5).astype(int)

# Evaluate the final model
accuracy_final_nn_top = accuracy_score(y_test, y_pred_final_nn)
report_final_nn_top = classification_report(y_test, y_pred_final_nn)

print(f'Position: top')
print(f'Selected Features: {selected_features_nn}')
print(f'Best Accuracy: {best_accuracy_nn}')
print('Final Model Accuracy:', accuracy_final_nn_top)
print('Classification Report:')
print(report_final_nn_top)
print('\n')

In [ ]:
# Filter the dataset to only include the 'jng' position
jungle_dataset = datasets['jng']

# Select only numeric columns
numeric_dataset = jungle_dataset.select_dtypes(include=['number'])

# Define the feature columns and target variable
features = numeric_dataset.drop(columns=['result'])
target = numeric_dataset['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Run backward selection
selected_features_nn, best_accuracy_nn = backward_selection_nn(X_train, y_train, X_test, y_test)

# Fit the final neural network model with selected features
scaler = StandardScaler()
X_train_scaled_selected = scaler.fit_transform(X_train[selected_features_nn])
X_test_scaled_selected = scaler.transform(X_test[selected_features_nn])

model_final_nn = Sequential()
model_final_nn.add(Dense(64, input_dim=X_train_scaled_selected.shape[1], activation='relu'))
model_final_nn.add(Dense(32, activation='relu'))
model_final_nn.add(Dense(1, activation='sigmoid'))

# Compile the model
model_final_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_final_nn.fit(X_train_scaled_selected, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Make predictions on the test set
y_pred_prob_final_nn = model_final_nn.predict(X_test_scaled_selected)
y_pred_final_nn = (y_pred_prob_final_nn > 0.5).astype(int)

# Evaluate the final model
accuracy_final_nn_jungle = accuracy_score(y_test, y_pred_final_nn)
report_final_nn_jungle = classification_report(y_test, y_pred_final_nn)

print(f'Position: jungle')
print(f'Selected Features: {selected_features_nn}')
print(f'Best Accuracy: {best_accuracy_nn}')
print('Final Model Accuracy:', accuracy_final_nn_jungle)
print('Classification Report:')
print(report_final_nn_jungle)
print('\n')

In [ ]:
# Filter the dataset to only include the 'mid' position
mid_dataset = datasets['mid']

# Select only numeric columns
numeric_dataset = mid_dataset.select_dtypes(include=['number'])

# Define the feature columns and target variable
features = numeric_dataset.drop(columns=['result'])
target = numeric_dataset['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Run backward selection
selected_features_nn, best_accuracy_nn = backward_selection_nn(X_train, y_train, X_test, y_test)

# Fit the final neural network model with selected features
scaler = StandardScaler()
X_train_scaled_selected = scaler.fit_transform(X_train[selected_features_nn])
X_test_scaled_selected = scaler.transform(X_test[selected_features_nn])

model_final_nn = Sequential()
model_final_nn.add(Dense(64, input_dim=X_train_scaled_selected.shape[1], activation='relu'))
model_final_nn.add(Dense(32, activation='relu'))
model_final_nn.add(Dense(1, activation='sigmoid'))

# Compile the model
model_final_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_final_nn.fit(X_train_scaled_selected, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Make predictions on the test set
y_pred_prob_final_nn = model_final_nn.predict(X_test_scaled_selected)
y_pred_final_nn = (y_pred_prob_final_nn > 0.5).astype(int)

# Evaluate the final model
accuracy_final_nn_mid = accuracy_score(y_test, y_pred_final_nn)
report_final_nn_mid = classification_report(y_test, y_pred_final_nn)

print(f'Position: mid')
print(f'Selected Features: {selected_features_nn}')
print(f'Best Accuracy: {best_accuracy_nn}')
print('Final Model Accuracy:', accuracy_final_nn_mid)
print('Classification Report:')
print(report_final_nn_mid)
print('\n')

In [ ]:
# Filter the dataset to only include the 'bot' position
bot_dataset = datasets['bot']

# Select only numeric columns
numeric_dataset = bot_dataset.select_dtypes(include=['number'])

# Define the feature columns and target variable
features = numeric_dataset.drop(columns=['result'])
target = numeric_dataset['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Run backward selection
selected_features_nn, best_accuracy_nn = backward_selection_nn(X_train, y_train, X_test, y_test)

# Fit the final neural network model with selected features
scaler = StandardScaler()
X_train_scaled_selected = scaler.fit_transform(X_train[selected_features_nn])
X_test_scaled_selected = scaler.transform(X_test[selected_features_nn])

model_final_nn = Sequential()
model_final_nn.add(Dense(64, input_dim=X_train_scaled_selected.shape[1], activation='relu'))
model_final_nn.add(Dense(32, activation='relu'))
model_final_nn.add(Dense(1, activation='sigmoid'))

# Compile the model
model_final_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_final_nn.fit(X_train_scaled_selected, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Make predictions on the test set
y_pred_prob_final_nn = model_final_nn.predict(X_test_scaled_selected)
y_pred_final_nn = (y_pred_prob_final_nn > 0.5).astype(int)

# Evaluate the final model
accuracy_final_nn_bot = accuracy_score(y_test, y_pred_final_nn)
report_final_nn_bot = classification_report(y_test, y_pred_final_nn)

print(f'Position: bot')
print(f'Selected Features: {selected_features_nn}')
print(f'Best Accuracy: {best_accuracy_nn}')
print('Final Model Accuracy:', accuracy_final_nn_bot)
print('Classification Report:')
print(report_final_nn_bot)
print('\n')

In [ ]:
# Filter the dataset to only include the 'sup' position
support_dataset = datasets['sup']

# Select only numeric columns
numeric_dataset = support_dataset.select_dtypes(include=['number'])

# Define the feature columns and target variable
features = numeric_dataset.drop(columns=['result'])
target = numeric_dataset['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Run backward selection
selected_features_nn, best_accuracy_nn = backward_selection_nn(X_train, y_train, X_test, y_test)

# Fit the final neural network model with selected features
scaler = StandardScaler()
X_train_scaled_selected = scaler.fit_transform(X_train[selected_features_nn])
X_test_scaled_selected = scaler.transform(X_test[selected_features_nn])

model_final_nn = Sequential()
model_final_nn.add(Dense(64, input_dim=X_train_scaled_selected.shape[1], activation='relu'))
model_final_nn.add(Dense(32, activation='relu'))
model_final_nn.add(Dense(1, activation='sigmoid'))

# Compile the model
model_final_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_final_nn.fit(X_train_scaled_selected, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Make predictions on the test set
y_pred_prob_final_nn = model_final_nn.predict(X_test_scaled_selected)
y_pred_final_nn = (y_pred_prob_final_nn > 0.5).astype(int)

# Evaluate the final model
accuracy_final_nn_sup = accuracy_score(y_test, y_pred_final_nn)
report_final_nn_sup = classification_report(y_test, y_pred_final_nn)

print(f'Position: support')
print(f'Selected Features: {selected_features_nn}')
print(f'Best Accuracy: {best_accuracy_nn}')
print('Final Model Accuracy:', accuracy_final_nn_sup)
print('Classification Report:')
print(report_final_nn_sup)
print('\n')

In [ ]:
# Define a dictionary to store the accuracy scores of all models
model_accuracies_complete = {
    'Logistic Regression': log_accuracy,
    'Random Forest': accuracy_rf_final,
    'KNN': accuracy_knn_final,
    'Neural Network': accuracy_final_nn,
}

# Find the model with the highest accuracy
best_model = max(model_accuracies_complete, key=model_accuracies_complete.get)
best_accuracy = model_accuracies_complete[best_model]

print(f'The best model is: {best_model}')
print(f'Accuracy: {best_accuracy}')

In [ ]:
# Define a dictionary to store the accuracy scores of all models for team wins
model_accuracies_teams = {
    'Logistic Regression': accuracy_teams,
    'Random Forest': accuracy_rf_final_teams,
    'KNN': accuracy_knn_final_teams,
    'Neural Network': accuracy_final_nn_teams,
}

# Find the model with the highest accuracy for team wins
best_model_teams = max(model_accuracies_teams, key=model_accuracies_teams.get)
best_accuracy_teams = model_accuracies_teams[best_model_teams]

print(f'The best model for predicting team wins is: {best_model_teams}')
print(f'Accuracy: {best_accuracy_teams}')

In [ ]:
# Define a dictionary to store the accuracy scores of all models for top lane wins
model_accuracies_top = {
    'Logistic Regression': best_models['top']['final_model_accuracy'],
    'Random Forest': best_models_rf['top']['final_model_accuracy'],
    'KNN': best_models_knn['top']['final_model_accuracy'],
    'Neural Network': accuracy_final_nn_top,
}

# Find the model with the highest accuracy for top lane wins
best_model_top = max(model_accuracies_top, key=model_accuracies_top.get)
best_accuracy_top = model_accuracies_top[best_model_top]

print(f'The best model for predicting top lane wins is: {best_model_top}')
print(f'Accuracy: {best_accuracy_top}')

In [ ]:
# Define a dictionary to store the accuracy scores of all models for jungle wins
model_accuracies_jungle = {
    'Logistic Regression': best_models['jng']['final_model_accuracy'],
    'Random Forest': best_models_rf['jng']['final_model_accuracy'],
    'KNN': best_models_knn['jng']['final_model_accuracy'],
    'Neural Network': accuracy_final_nn_jungle,
}

# Find the model with the highest accuracy for jungle wins
best_model_jungle = max(model_accuracies_jungle, key=model_accuracies_jungle.get)
best_accuracy_jungle = model_accuracies_jungle[best_model_jungle]

print(f'The best model for predicting jungle wins is: {best_model_jungle}')
print(f'Accuracy: {best_accuracy_jungle}')

In [ ]:
# Define a dictionary to store the accuracy scores of all models for mid lane wins
model_accuracies_mid = {
    'Logistic Regression': best_models['mid']['final_model_accuracy'],
    'Random Forest': best_models_rf['mid']['final_model_accuracy'],
    'KNN': best_models_knn['mid']['final_model_accuracy'],
    'Neural Network': accuracy_final_nn_mid,
}

# Find the model with the highest accuracy for mid lane wins
best_model_mid = max(model_accuracies_mid, key=model_accuracies_mid.get)
best_accuracy_mid = model_accuracies_mid[best_model_mid]

print(f'The best model for predicting mid lane wins is: {best_model_mid}')
print(f'Accuracy: {best_accuracy_mid}')

In [ ]:
# Define a dictionary to store the accuracy scores of all models for bot lane wins
model_accuracies_bot = {
    'Logistic Regression': best_models['bot']['final_model_accuracy'],
    'Random Forest': best_models_rf['bot']['final_model_accuracy'],
    'KNN': best_models_knn['bot']['final_model_accuracy'],
    'Neural Network': accuracy_final_nn_bot,
}

# Find the model with the highest accuracy for bot lane wins
best_model_bot = max(model_accuracies_bot, key=model_accuracies_bot.get)
best_accuracy_bot = model_accuracies_bot[best_model_bot]

print(f'The best model for predicting bot lane wins is: {best_model_bot}')
print(f'Accuracy: {best_accuracy_bot}')

In [ ]:
# Define a dictionary to store the accuracy scores of all models for support wins
model_accuracies_sup = {
    'Logistic Regression': best_models['sup']['final_model_accuracy'],
    'Random Forest': best_models_rf['sup']['final_model_accuracy'],
    'KNN': best_models_knn['sup']['final_model_accuracy'],
    'Neural Network': accuracy_final_nn_sup,
}

# Find the model with the highest accuracy for support wins
best_model_sup = max(model_accuracies_sup, key=model_accuracies_sup.get)
best_accuracy_sup = model_accuracies_sup[best_model_sup]

print(f'The best model for predicting support wins is: {best_model_sup}')
print(f'Accuracy: {best_accuracy_sup}')